In [3]:
import requests
import json
import pandas as pd

In [4]:
API_key = "81f1a5b752c4569a954655d748508db5"
cities = ["Berlin"]
country_code = "DE"

In [102]:
for city in range(len(cities)):
    url = f" https://api.openweathermap.org/data/2.5/forecast?q={cities[city]},{country_code}&appid={API_key}&units=metric"
    openweather_raw = requests.get(url).json()

In [33]:
openweather_raw.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

Static City Informations

In [19]:
openweather_raw["city"]


{'id': 2950159,
 'name': 'Berlin',
 'coord': {'lat': 52.5244, 'lon': 13.4105},
 'country': 'DE',
 'population': 1000000,
 'timezone': 3600,
 'sunrise': 1709789928,
 'sunset': 1709830567}

In [20]:
openweather_raw["city"]["name"]

'Berlin'

In [22]:
openweather_raw["city"]["coord"]["lat"]

52.5244

In [23]:
openweather_raw["city"]["coord"]["lon"]

13.4105

In [25]:
openweather_raw["city"]["country"]

'DE'

In [113]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

API_key = "81f1a5b752c4569a954655d748508db5"
cities = ["Berlin", "München"]
country_code = "DE"
cities_dict = {"city_id": [], "city": [], "country": [],"population": [], "lat": [], "lon": [], "elevation_[m]": []}

for city in cities:

    url = f" https://api.openweathermap.org/data/2.5/forecast?q={city},{country_code}&appid={API_key}&units=metric"
    openweather_raw = requests.get(url).json()

    cities_dict["city_id"].append(openweather_raw["city"]["id"])
    cities_dict["city"].append(openweather_raw["city"]["name"])
    cities_dict["country"].append(openweather_raw["city"]["country"])
    cities_dict["lat"].append(openweather_raw["city"]["coord"]["lat"])
    cities_dict["lon"].append(openweather_raw["city"]["coord"]["lon"])

    #webscrape
    response = requests.get(f"https://en.wikipedia.org/wiki/{city}")
    soup = BeautifulSoup(response.content, 'html.parser')
    infobox = soup.find("table", {"class": "infobox ib-settlement vcard"})

    if infobox:
        # Search for the header containing population information
        population_data = infobox.find(string="Population")

        # Extract the population value if found, format number and add to list
        if population_data:
            population = population_data.find_next("td").get_text().strip().replace(",", "")
            cities_dict["population"].append(population)
        else:
            print(f"Population data not found for {city}.")

        # Search for elevation, extract and add to list
        elevation = infobox.find(string="Elevation")
        if elevation:
            elevation = elevation.find_next("td").get_text().strip()
            elevation = int(re.sub(r'\D', '', elevation[:5]))
            cities_dict["elevation_[m]"].append(elevation)
        else:
            elevation_list.append(None)
    else:
        print(f"Infobox not found for {city}.")

    
cities_df = pd.DataFrame(cities_dict)
cities_df    

,city_id,city,country,population,lat,lon,elevation_[m]
0,2950159,Berlin,DE,3576873,52.5244,13.4105,34
1,2867714,Munich,DE,1512491,48.1374,11.5755,520


Dynamic Weather Information


In [58]:
openweather_raw

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1709845200,
   'main': {'temp': 4.13,
    'feels_like': 1.39,
    'temp_min': 3.24,
    'temp_max': 4.13,
    'pressure': 1012,
    'sea_level': 1012,
    'grnd_level': 1020,
    'humidity': 64,
    'temp_kf': 0.89},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}],
   'clouds': {'all': 0},
   'wind': {'speed': 3.1, 'deg': 93, 'gust': 8.07},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2024-03-07 21:00:00'},
  {'dt': 1709856000,
   'main': {'temp': 2.95,
    'feels_like': 0.04,
    'temp_min': 2.13,
    'temp_max': 2.95,
    'pressure': 1019,
    'sea_level': 1019,
    'grnd_level': 1019,
    'humidity': 67,
    'temp_kf': 0.82},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}],
   'clouds': {'all': 0},
   'wind': {'speed': 3, 'deg': 88, 'gust': 7.84},
   'visibility': 10000,
   'pop': 0,
   

In [95]:
openweather_raw["list"][0]["dt_txt"].info()

AttributeError: 'str' object has no attribute 'info'

In [90]:
openweather_raw["list"][0]["rain"]["3h"]

0

In [115]:
import json
import pandas as pd
from datetime import datetime

API_key = "81f1a5b752c4569a954655d748508db5"
cities = ["Berlin", "München"]
country_code = "DE"
weather_dict = {
    "city_id": [],
    "temp_[°C]": [],
    "temp_(feels_like)_[°C]": [], 
    "main_weather": [], 
    "wind_speed_[m/s]": [], 
    "gust_speed_[m/s]": [], 
    "sight_distance_[m]": [], 
    "probability_of_precipitation_[%]": [], 
    "rain_last_3h_[mm]": [], 
    "snow_last_3h_[mm]": [], 
    "forecast_time": [], 
    "timestamp": [] }

for city in cities:

    url = f" https://api.openweathermap.org/data/2.5/forecast?q={city},{country_code}&appid={API_key}&units=metric"
    openweather_raw = requests.get(url).json()

    for i in range(openweather_raw["cnt"]):
        weather_dict["city_id"].append(openweather_raw["city"]["id"])
        weather_dict["temp_[°C]"].append(openweather_raw["list"][i]["main"]["temp"])
        weather_dict["temp_(feels_like)_[°C]"].append(openweather_raw["list"][i]["main"]["feels_like"])
        weather_dict["main_weather"].append(openweather_raw["list"][i]["weather"][0]["main"])
        weather_dict["wind_speed_[m/s]"].append(openweather_raw["list"][i]["wind"]["speed"])
        weather_dict["gust_speed_[m/s]"].append(openweather_raw["list"][i]["wind"]["gust"])
        weather_dict["sight_distance_[m]"].append(openweather_raw["list"][i]["visibility"])
        weather_dict["probability_of_precipitation_[%]"].append(openweather_raw["list"][i]["pop"]*100)
        try:
            weather_dict["rain_last_3h_[mm]"].append(openweather_raw["list"][i]["rain"]["3h"])
        except:
            weather_dict["rain_last_3h_[mm]"].append(0)
        try:
            weather_dict["snow_last_3h_[mm]"].append(openweather_raw["list"][i]["snow"]["3h"])
        except:
            weather_dict["snow_last_3h_[mm]"].append(0)
        weather_dict["forecast_time"].append(pd.to_datetime(openweather_raw["list"][i]["dt_txt"]))
        weather_dict["timestamp"].append(pd.to_datetime(datetime.now().replace(microsecond=0)))
        

weather_df = pd.DataFrame(weather_dict)    
weather_df

,city_id,temp_[°C],temp_(feels_like)_[°C],main_weather,wind_speed_[m/s],gust_speed_[m/s],sight_distance_[m],robability_of_precipitation_[%],rain_last_3h_[mm],snow_last_3h_[mm],forecast_time,timestamp
0,2950159,8.39,5.60,Clear,4.94,6.65,10000,0.0,0.0,0,2024-03-08 12:00:00,2024-03-08 12:43:42
1,2950159,8.52,5.75,Clear,4.96,6.93,10000,0.0,0.0,0,2024-03-08 15:00:00,2024-03-08 12:43:42
2,2950159,6.55,3.42,Clear,4.68,10.61,10000,0.0,0.0,0,2024-03-08 18:00:00,2024-03-08 12:43:42
3,2950159,3.93,0.22,Clear,4.57,12.21,10000,0.0,0.0,0,2024-03-08 21:00:00,2024-03-08 12:43:42
4,2950159,2.66,-0.89,Clear,3.81,9.64,10000,0.0,0.0,0,2024-03-09 00:00:00,2024-03-08 12:43:42
...,...,...,...,...,...,...,...,...,...,...,...,...
75,2867714,4.53,1.89,Clouds,3.07,8.34,10000,0.0,0.0,0,2024-03-12 21:00:00,2024-03-08 12:43:43
76,2867714,3.31,0.51,Clouds,2.96,7.65,10000,0.0,0.0,0,2024-03-13 00:00:00,2024-03-08 12:43:43
77,2867714,2.13,-0.56,Clouds,2.56,5.77,10000,0.0,0.0,0,2024-03-13 03:00:00,2024-03-08 12:43:43
78,2867714,2.33,-0.11,Clouds,2.34,4.91,10000,0.0,0.0,0,2024-03-13 06:00:00,2024-03-08 12:43:43


# Automatic SQL Push

### 1. city data to sql

In [292]:
def push_city_data_to_sql():

   import pandas as pd
   import requests
   from bs4 import BeautifulSoup
   import re
   from sqlalchemy import create_engine, inspect, Float

   API_key = "81f1a5b752c4569a954655d748508db5"
   cities = ["Berlin", "München", "Hamburg", "Bremen"]
   country_code = "DE"
   cities_dict = {"city_id": [], "city": [], "country": [],"population": [], "lat": [], "lon": [], "elevation_[m]": []}

   for city in cities:

      url = f" https://api.openweathermap.org/data/2.5/forecast?q={city},{country_code}&appid={API_key}&units=metric"
      openweather_raw = requests.get(url).json()

      if openweather_raw["cod"] == "200":

         cities_dict["city_id"].append(openweather_raw["city"]["id"])
         cities_dict["city"].append(openweather_raw["city"]["name"])
         cities_dict["country"].append(openweather_raw["city"]["country"])
         cities_dict["lat"].append(openweather_raw["city"]["coord"]["lat"])
         cities_dict["lon"].append(openweather_raw["city"]["coord"]["lon"])

         #webscrape
         response = requests.get(f"https://en.wikipedia.org/wiki/{city}")
         soup = BeautifulSoup(response.content, 'html.parser')
         infobox = soup.find("table", {"class": "infobox ib-settlement vcard"})

         if infobox:
            # Search for the header containing population information
            population_data = infobox.find(string="Population")

            # Extract the population value if found, format number and add to list
            if population_data:
                  population = population_data.find_next("td").get_text().strip().replace(",", "")
                  cities_dict["population"].append(population)
            else:
                  print(f"Population data not found for {city}.")

            # Search for elevation, extract and add to list
            elevation = infobox.find(string="Elevation")
            if elevation:
                  elevation = elevation.find_next("td").get_text().strip()
                  elevation = int(re.sub(r'\D', '', elevation[:5]))
                  cities_dict["elevation_[m]"].append(elevation)
            else:
                  cities_dict["elevation_[m]"].append(None)
         else:
            print(f"Infobox not found for {city}.")

      else: 
         print(f"error code {openweather_raw['cod']} in query for {city}")

   cities_df = pd.DataFrame(cities_dict)

   cities_df["population"].astype(int)

   schema = "gans"
   host = "127.0.0.1"
   user = "root"
   password = "MySQLjau"
   port = 3306
   connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

   engine = create_engine(connection_string)
   inspector = inspect(engine)
   if 'cities' in inspector.get_table_names():
      existing_data = pd.read_sql('cities', engine)
      cities_df = cities_df[~cities_df['city_id'].isin(existing_data['city_id'])]
   
   cities_df.to_sql('cities',
                  con=engine,
                  if_exists='append',
                  index=False,
                  dtype={'lat': Float, 'lon': Float}
                  )
   
   return cities_df


In [294]:
push_city_data_to_sql()

,city_id,city,country,population,lat,lon,elevation_[m]
0,2950159,Berlin,DE,3576873,52.5244,13.4105,34.0
1,2867714,Munich,DE,1512491,48.1374,11.5755,520.0
2,2911298,Hamburg,DE,1945532,53.5500,10.0000,NaN
3,2944388,Bremen,DE,563290,53.0752,8.8078,12.0


In [277]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "MySQLjau"
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
pd.read_sql("cities",con=connection_string).dtypes

city_id            int64
city              object
country           object
population         int64
lat              float64
lon              float64
elevation_[m]    float64
dtype: object

### 2. weather data to sql

In [266]:
def push_weather_data_to_sql():

    import pandas as pd
    from datetime import datetime
    import requests

    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = "MySQLjau"
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    API_key = "81f1a5b752c4569a954655d748508db5"
    cities = (pd.read_sql("cities", con=connection_string))["city"]
    country_code = (pd.read_sql("cities", con=connection_string))["country"]
    weather_dict = {
        "city_id": [],
        "temp_[°C]": [],
        "temp_(feels_like)_[°C]": [], 
        "main_weather": [], 
        "wind_speed_[m/s]": [], 
        "gust_speed_[m/s]": [], 
        "sight_distance_[m]": [], 
        "probability_of_precipitation_[%]": [], 
        "rain_last_3h_[mm]": [], 
        "snow_last_3h_[mm]": [], 
        "forecast_time": [], 
        "timestamp": [] }
    

    for city in cities:

        url = f" https://api.openweathermap.org/data/2.5/forecast?q={city},{country_code}&appid={API_key}&units=metric"
        openweather_raw = requests.get(url).json()
        
        if openweather_raw["cod"] == "200":

            for i in range(openweather_raw["cnt"]):
                weather_dict["city_id"].append(openweather_raw["city"]["id"])
                weather_dict["temp_[°C]"].append(openweather_raw["list"][i]["main"]["temp"])
                weather_dict["temp_(feels_like)_[°C]"].append(openweather_raw["list"][i]["main"]["feels_like"])
                weather_dict["main_weather"].append(openweather_raw["list"][i]["weather"][0]["main"])
                weather_dict["wind_speed_[m/s]"].append(openweather_raw["list"][i]["wind"]["speed"])
                weather_dict["gust_speed_[m/s]"].append(openweather_raw["list"][i]["wind"]["gust"])
                weather_dict["sight_distance_[m]"].append(openweather_raw["list"][i]["visibility"])
                weather_dict["probability_of_precipitation_[%]"].append(openweather_raw["list"][i]["pop"]*100)
                try:
                    weather_dict["rain_last_3h_[mm]"].append(openweather_raw["list"][i]["rain"]["3h"])
                except:
                    weather_dict["rain_last_3h_[mm]"].append(0)
                try:
                    weather_dict["snow_last_3h_[mm]"].append(openweather_raw["list"][i]["snow"]["3h"])
                except:
                    weather_dict["snow_last_3h_[mm]"].append(0)
                weather_dict["forecast_time"].append(pd.to_datetime(openweather_raw["list"][i]["dt_txt"]))
                weather_dict["timestamp"].append(pd.to_datetime(datetime.now().replace(microsecond=0)))
            
        else: 
                print(f"error code {openweather_raw['cod']} in query for {city}")
                
    weather_df = pd.DataFrame(weather_dict)

    weather_df["snow_last_3h_[mm]"].astype(float)
    
    weather_df.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)

    return weather_df

In [257]:
push_weather_data_to_sql().dtypes

id                                           int64
city_id                                      int64
temp_[°C]                                  float64
temp_(feels_like)_[°C]                     float64
main_weather                                object
wind_speed_[m/s]                           float64
gust_speed_[m/s]                           float64
sight_distance_[m]                           int64
probability_of_precipitation_[%]           float64
rain_last_3h_[mm]                          float64
snow_last_3h_[mm]                            int64
forecast_time                       datetime64[ns]
timestamp                           datetime64[ns]
dtype: object

In [248]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "MySQLjau"
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

pd.read_sql("cities", con=connection_string).dtypes

city_id            int64
city              object
country           object
population         int64
lat              float64
lon              float64
elevation_[m]    float64
dtype: object

In [252]:
pd.read_sql("weather", con=connection_string).dtypes

id                                           int64
city_id                                      int64
temp_[°C]                                  float64
temp_(feels_like)_[°C]                     float64
main_weather                                object
wind_speed_[m/s]                           float64
gust_speed_[m/s]                           float64
sight_distance_[m]                           int64
probability_of_precipitation_[%]           float64
rain_last_3h_[mm]                          float64
snow_last_3h_[mm]                            int64
forecast_time                       datetime64[ns]
timestamp                           datetime64[ns]
dtype: object

In [253]:
weather_df.dtypes

city_id                                     int64
temp_[°C]                                 float64
temp_(feels_like)_[°C]                    float64
main_weather                               object
wind_speed_[m/s]                          float64
gust_speed_[m/s]                          float64
sight_distance_[m]                          int64
robability_of_precipitation_[%]           float64
rain_last_3h_[mm]                         float64
snow_last_3h_[mm]                           int64
forecast_time                      datetime64[ns]
timestamp                          datetime64[ns]
dtype: object

In [203]:
openweather_raw["cnt"]

40

In [268]:
push_weather_data_to_sql()

,city_id,temp_[°C],temp_(feels_like)_[°C],main_weather,wind_speed_[m/s],gust_speed_[m/s],sight_distance_[m],probability_of_precipitation_[%],rain_last_3h_[mm],snow_last_3h_[mm],forecast_time,timestamp
0,2867714,11.52,10.19,Clouds,1.11,1.67,10000,0.0,0.00,0,2024-03-11 12:00:00,2024-03-11 11:57:51
1,2867714,11.55,10.12,Clouds,1.44,1.85,10000,6.0,0.00,0,2024-03-11 15:00:00,2024-03-11 11:57:51
2,2867714,9.19,8.71,Clouds,1.51,1.74,10000,4.0,0.00,0,2024-03-11 18:00:00,2024-03-11 11:57:51
3,2867714,6.61,5.78,Clouds,1.50,2.71,10000,4.0,0.00,0,2024-03-11 21:00:00,2024-03-11 11:57:51
4,2867714,5.24,4.00,Clouds,1.68,2.30,10000,4.0,0.00,0,2024-03-12 00:00:00,2024-03-11 11:57:51
...,...,...,...,...,...,...,...,...,...,...,...,...
155,2950159,12.21,11.60,Rain,3.06,8.52,10000,20.0,0.20,0,2024-03-15 21:00:00,2024-03-11 11:57:51
156,2950159,10.83,10.27,Rain,3.01,8.40,10000,28.0,0.30,0,2024-03-16 00:00:00,2024-03-11 11:57:51
157,2950159,10.68,10.10,Rain,2.94,8.01,10000,27.0,0.13,0,2024-03-16 03:00:00,2024-03-11 11:57:51
158,2950159,10.85,10.29,Rain,4.03,9.88,10000,44.0,0.60,0,2024-03-16 06:00:00,2024-03-11 11:57:51
